In [ ]:
# ARIEL DATA CHALLENGE 2025 - DAY 5 RECONNAISSANCE
# Transitioning Day 4 Synthetic Framework to Real Competition Data
# Target: Map proven multi-visit ensemble to 270GB real dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("🚀 ARIEL DATA CHALLENGE 2025 - REAL DATA RECONNAISSANCE")
print("=" * 60)
print("Mission: Adapt Day 4 framework to championship dataset")
print("Target: Multi-visit noise reduction + physics-informed features")

In [ ]:

# =============================================================================
# PHASE 1: DATA LANDSCAPE MAPPING
# =============================================================================

data_path = Path("/kaggle/input/ariel-data-challenge-2025")
print(f"\n📊 DATASET INVENTORY:")
print("-" * 40)

total_size = 0
file_count = 0
for item in sorted(data_path.glob("*")):
    if item.is_file():
        size_mb = item.stat().st_size / (1024*1024)
        total_size += size_mb
        file_count += 1
        print(f"  {item.name:<25} {size_mb:>8.1f} MB")

print(f"\nTotal: {file_count} files, {total_size/1024:.1f} GB")


In [ ]:

# =============================================================================
# PHASE 2: METADATA INTELLIGENCE
# =============================================================================

print(f"\n🎯 COMPETITION PARAMETERS:")
print("-" * 40)

# Load core metadata
train_df = pd.read_csv(data_path / "train.csv")
wavelengths_df = pd.read_csv(data_path / "wavelengths.csv")
axis_info_df = pd.read_parquet(data_path / "axis_info.parquet")
adc_info_df = pd.read_csv(data_path / "adc_info.csv")
train_star_info = pd.read_csv(data_path / "train_star_info.csv")

print(f"Training planets: {len(train_df)}")
print(f"Wavelength grid: {len(wavelengths_df)} points")
print(f"Ground truth spectrum shape: {train_df.iloc[:, 1:].shape}")
print(f"Star parameters: {len(train_star_info)} systems")

# Examine ground truth structure
gt_spectra = train_df.iloc[:, 1:].values
print(f"\nGround truth analysis:")
print(f"  Spectrum length: {gt_spectra.shape[1]} wavelengths")
print(f"  Value range: [{gt_spectra.min():.6f}, {gt_spectra.max():.6f}]")
print(f"  Mean signal: {gt_spectra.mean():.6f}")

In [ ]:
# =============================================================================
# PHASE 3: MULTI-VISIT OPPORTUNITY ASSESSMENT
# =============================================================================

print(f"\n🔄 MULTI-VISIT FRAMEWORK VALIDATION:")
print("-" * 40)

train_path = data_path / "train"
planet_dirs = list(train_path.glob("*"))[:10]  # Sample first 10

multi_visit_stats = {"single_visit": 0, "multi_visit": 0, "max_visits": 0}

for planet_path in planet_dirs:
    planet_id = planet_path.name
    fgs1_files = list(planet_path.glob("FGS1_signal_*.parquet"))
    airs_files = list(planet_path.glob("AIRS-CH0_signal_*.parquet"))
    
    total_visits = len(fgs1_files) + len(airs_files)
    
    if total_visits > 2:
        multi_visit_stats["multi_visit"] += 1
        multi_visit_stats["max_visits"] = max(multi_visit_stats["max_visits"], total_visits)
        print(f"  🎯 {planet_id}: {len(fgs1_files)} FGS1 + {len(airs_files)} AIRS = {total_visits} total obs")
    else:
        multi_visit_stats["single_visit"] += 1

print(f"\nMulti-visit summary (sample of {len(planet_dirs)} planets):")
print(f"  Single visit: {multi_visit_stats['single_visit']}")
print(f"  Multi-visit: {multi_visit_stats['multi_visit']} ← YOUR ADVANTAGE!")
print(f"  Max visits: {multi_visit_stats['max_visits']}")

In [ ]:
# =============================================================================
# PHASE 4: INSTRUMENT SPECIFICATION MAPPING
# =============================================================================

print(f"\n📡 INSTRUMENT ARCHITECTURE:")
print("-" * 40)

print("FGS1 (Fine Guidance System):")
print(f"  Wavelength: 0.60-0.80 μm (visible)")
print(f"  Time steps: 0.1 seconds")
print(f"  Frames: 135,000 per observation")
print(f"  Image size: 32×32 pixels (1,024 total)")

print("\nAIRS-CH0 (Infrared Spectrometer):")
print(f"  Wavelength: 1.95-3.90 μm (infrared)")
print(f"  Frames: 11,250 per observation") 
print(f"  Image size: 32×356 pixels (11,392 total)")

# ADC correction parameters
print(f"\nADC Correction Parameters:")
for col in adc_info_df.columns:
    val = adc_info_df[col].iloc[0]
    print(f"  {col}: {val}")

In [ ]:
# =============================================================================
# PHASE 5: WAVELENGTH GRID ANALYSIS
# =============================================================================

print(f"\n🌈 WAVELENGTH TARGETING:")
print("-" * 40)

wavelength_grid = wavelengths_df.values.flatten()
print(f"Wavelength range: {wavelength_grid.min():.3f} - {wavelength_grid.max():.3f} μm")
print(f"Grid resolution: {len(wavelength_grid)} points")

# Your Day 4 H2O targeting vs real data
h2o_bands = [1.4, 1.9, 2.7]
print(f"\nH2O absorption band mapping:")
print(f"Day 4 targets: {h2o_bands} μm")

for band in h2o_bands:
    # Find closest wavelengths
    distances = np.abs(wavelength_grid - band)
    closest_idx = np.argmin(distances)
    closest_wl = wavelength_grid[closest_idx]
    
    # Check if in reasonable range (±0.2 μm)
    if distances[closest_idx] < 0.2:
        print(f"  ✅ {band} μm → index {closest_idx} (actual: {closest_wl:.3f} μm)")
    else:
        print(f"  ❌ {band} μm → No close match (closest: {closest_wl:.3f} μm)")

# Check which instrument covers which H2O bands
print(f"\nInstrument coverage for H2O bands:")
for band in h2o_bands:
    if 1.95 <= band <= 3.90:
        print(f"  {band} μm: AIRS-CH0 ✅")
    elif 0.60 <= band <= 0.80:
        print(f"  {band} μm: FGS1 ✅")
    else:
        print(f"  {band} μm: Neither instrument ❌")

In [ ]:
# =============================================================================
# PHASE 6: SAMPLE DATA LOADING TEST
# =============================================================================

print(f"\n🧪 SAMPLE DATA LOADING TEST:")
print("-" * 40)

def load_planet_observations(planet_id, instrument="FGS1"):
    """Load all observations for a planet - testing your multi-visit framework"""
    planet_path = train_path / planet_id
    
    if instrument == "FGS1":
        pattern = "FGS1_signal_*.parquet"
        expected_frames = 135000
        image_shape = (32, 32)
    else:  # AIRS-CH0
        pattern = "AIRS-CH0_signal_*.parquet"
        expected_frames = 11250
        image_shape = (32, 356)
    
    observations = []
    for file_path in sorted(planet_path.glob(pattern)):
        print(f"    Loading {file_path.name}...")
        data = pd.read_parquet(file_path).values
        
        # Apply ADC correction (restore dynamic range)
        gain = adc_info_df[f"{instrument}_adc_gain"].iloc[0]
        offset = adc_info_df[f"{instrument}_adc_offset"].iloc[0]
        corrected_data = data * gain + offset
        
        print(f"      Shape: {corrected_data.shape}")
        print(f"      Range: [{corrected_data.min():.2f}, {corrected_data.max():.2f}]")
        
        observations.append(corrected_data)
    
    return observations

# Test on first planet with multiple observations
test_planet = None
for planet_path in planet_dirs:
    fgs1_count = len(list(planet_path.glob("FGS1_signal_*.parquet")))
    if fgs1_count > 1:
        test_planet = planet_path.name
        break

if test_planet:
    print(f"Testing multi-visit loading on planet: {test_planet}")
    fgs1_obs = load_planet_observations(test_planet, "FGS1")
    
    print(f"\n🎯 MULTI-VISIT VALIDATION:")
    print(f"  Loaded {len(fgs1_obs)} FGS1 observations")
    
    if len(fgs1_obs) >= 2:
        # Quick noise reduction test (your Day 4 concept)
        obs1_flux = np.mean(fgs1_obs[0])
        obs2_flux = np.mean(fgs1_obs[1])
        combined_flux = (obs1_flux + obs2_flux) / 2
        
        # Estimate noise reduction
        obs1_std = np.std(fgs1_obs[0])
        obs2_std = np.std(fgs1_obs[1])
        theoretical_improvement = np.sqrt(2)  # √N for N=2 visits
        
        print(f"  Obs 1 mean flux: {obs1_flux:.2f} ± {obs1_std:.2f}")
        print(f"  Obs 2 mean flux: {obs2_flux:.2f} ± {obs2_std:.2f}")
        print(f"  Combined flux: {combined_flux:.2f}")
        print(f"  Theoretical √N improvement: {theoretical_improvement:.2f}x")
        print(f"  🚀 YOUR MULTI-VISIT FRAMEWORK IS APPLICABLE!")
else:
    print("No multi-visit planets found in sample - checking larger set...")